In [ ]:
import os
os.chdir("../")

In [1]:
import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras import regularizers

import keras.backend as K

Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [3]:
from utils import *
df = functions.parse_file(r"raw_data/EmoContext/train3.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,dont worry ism girl,hmm how do i know if you are,whats ur name ? ?,others
1,1,when did in i?,saw many times i think -_-,no . i never saw you .,angry
2,2,by,by google chrome,where you live,others
3,3,u r ridiculous,i might be ridiculous but i am telling the tru...,u little disgusting whore,angry
4,4,just for time pass,wt do u do a a living then 4,maybe,others


In [4]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [5]:
NR_WORDS = 8000
MAX_PROP_LENGTH = 128
tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

tokenizer.fit_on_texts(text_data)

X_train = tokenizer.texts_to_sequences(text_data)
X_train = pad_sequences(X_train, maxlen = MAX_PROP_LENGTH)

In [6]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_train = []

for idx,row in df.iterrows():
    Y_train.append(one_hot_vector(row['label']))

Y_train = np.array(Y_train)

In [7]:
df = functions.parse_file(r"raw_data/EmoContext/devwithlabels3.txt", "EmoContext")
df.head(5)

,id,turn1,turn2,turn3,label
0,0,then dont ask me,youre a guy not as if you would understand,im not a guy fuck off,angry
1,1,mixed things such as ??,the things you do . .,have you seen minions ??,others
2,2,today ism very happy,and ism happy for you,i will be marry,happy
3,3,noah bring me some,left it there oops,orb,others
4,4,it is thoo,i said soon master . .,he is pressuring me,others


In [8]:
text_data = []
for idx,row in df.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

In [9]:
#tokenizer = Tokenizer(num_words=NR_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
#                                   lower=True,split=' ')

#tokenizer.fit_on_texts(text_data)

X_test = tokenizer.texts_to_sequences(text_data)
X_test = pad_sequences(X_test, maxlen = MAX_PROP_LENGTH)

In [10]:
def one_hot_vector(word,label=None):
    words = {"others": 0, "angry": 1, "sad":2, "happy": 3}
    if label == None:
        y = [0,0,0,0]
        y[words[word]] = 1
        return y
    if label == word:
        return [1,0]
    return [0,1]

Y_test = []

for idx,row in df.iterrows():
    Y_test.append(one_hot_vector(row['label']))

Y_test = np.array(Y_test)

In [11]:
embed_dim = 256
lstm_out = 32
batch_size = 64

adam = optimizers.Adam(lr=0.01)
rmsprop = optimizers.RMSprop(lr=0.01, rho=0.95, epsilon=None, decay=0.0)


model = Sequential()
model.name = "simple_model"
model.add(Embedding(NR_WORDS, embed_dim,input_length = X_train.shape[1],name="simple_embeding_model",trainable=True))
model.add(LSTM(lstm_out))
model.add(Dense(4,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer=rmsprop, metrics = ['accuracy', f1])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_embeding_model (Embed (None, 128, 256)          2048000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                36992     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 132       
Total params: 2,085,124
Trainable params: 2,085,124
Non-trainable params: 0
_________________________________________________________________


In [13]:
mdcheck = ModelCheckpoint("trained_models/best_model_val_acc{val_acc:.4f}.h5", monitor='val_f1', save_best_only=True)

In [14]:
Y_train = np.array(Y_train)
total = len(Y_train)
history = model.fit(X_train, Y_train,
                    validation_data=(X_test, Y_test),
                    epochs=10, verbose=1, batch_size=batch_size,
                    class_weight={
                        0: total / len(np.where(Y_train[:,0]==1.0)[0]),
                        1: total / len(np.where(Y_train[:,1]==1.0)[0]),
                        2: total / len(np.where(Y_train[:,2]==1.0)[0]),
                        3: total / len(np.where(Y_train[:,3]==1.0)[0]),
                    },callbacks=[mdcheck])

Train on 30160 samples, validate on 2755 samples
Epoch 1/10
30160/30160 [==============================] - 193s 6ms/step - loss: 2.4438 - acc: 0.7626 - f1: 0.7332 - val_loss: 0.6992 - val_acc: 0.7593 - val_f1: 0.5574
Epoch 2/10
30160/30160 [==============================] - 124s 4ms/step - loss: 1.7900 - acc: 0.8278 - f1: 0.8128 - val_loss: 0.6630 - val_acc: 0.7775 - val_f1: 0.5751
Epoch 3/10
30160/30160 [==============================] - 68s 2ms/step - loss: 1.5628 - acc: 0.8466 - f1: 0.8348 - val_loss: 0.8162 - val_acc: 0.7024 - val_f1: 0.5245
Epoch 4/10
30160/30160 [==============================] - 102s 3ms/step - loss: 1.4128 - acc: 0.8639 - f1: 0.8526 - val_loss: 0.9287 - val_acc: 0.6588 - val_f1: 0.4902
Epoch 5/10
30160/30160 [==============================] - 107s 4ms/step - loss: 1.3345 - acc: 0.8726 - f1: 0.8606 - val_loss: 0.8383 - val_acc: 0.7024 - val_f1: 0.5274
Epoch 6/10
30160/30160 [==============================] - 104s 3ms/step - loss: 1.2801 - acc: 0.8781 - f1: 0.868

In [15]:
save_model(model,indices=[{"indices":["turn1","turn2","turn3"],"NR_WORDS":NR_WORDS,"MAX_PROP_LENGTH":MAX_PROP_LENGTH}])

In [16]:
df_test = functions.parse_file(r"raw_data/EmoContext/devwithlabels3.txt", "EmoContext")
df_test.head()

,id,turn1,turn2,turn3,label
0,0,then dont ask me,youre a guy not as if you would understand,im not a guy fuck off,angry
1,1,mixed things such as ??,the things you do . .,have you seen minions ??,others
2,2,today ism very happy,and ism happy for you,i will be marry,happy
3,3,noah bring me some,left it there oops,orb,others
4,4,it is thoo,i said soon master . .,he is pressuring me,others


In [17]:
text_data = []

for idx,row in df_test.iterrows():
    text_data.append("{}. {}. {}.".format(row['turn1'], row['turn2'], row['turn3']))

res = model.predict(X_test, batch_size=64, verbose=1)
res[3]

2755/2755 [==============================] - 2s 892us/step


array([7.5361764e-01, 2.1626547e-04, 1.6319562e-01, 8.2970411e-02],
      dtype=float32)

In [18]:
revers_words = {0:"others", 1:"angry", 2:"sad", 3:"happy"}

def softmax_convert(res):
    max_i = 0
    max_v = 0
    for i in range(0,4):
        if res[i] > max_v:
            max_v = res[i]
            max_i = i
    return revers_words[max_i]

In [19]:
results = []
for r in res:
    results.append(softmax_convert(r))
    
df_test['label'] = results
df_test.head(50)
df_test.to_csv("simple_model_poornes.txt",index=False , sep="\t")